In [1]:
import spotipy.util as util
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd

username = '1266336804'
client_id = '40a1fa542f6d4637a0154eae9994a109'
client_secret = 'dec63e56d9944dafa56f04c2aeb04f07'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-library-read'

token = util.prompt_for_user_token(username=username,
                                  scope=scope,
                                  client_id=client_id,
                                  client_secret=client_secret,
                                  redirect_uri=redirect_uri)

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token)

In [3]:
print(token)

BQAXkcPD9bO8rMhxEa-edhBcx0FaL0c5KF7i6f0zT0GNvsgY3EUKspaqwIkHA1j5kQZ0yp4RGNNPmNfnQKRhNe7VzVl4XG21JVjwCQrfrrVhgPzDX6EluermVksZWV2BTTv95MFMl-iNch-xo1CT6Zl_DNlOT3J1jA


In [4]:
import requests
def get_saved_songs(token: str, offset: int) -> str:  # : str are annotations for what the variables mean,
    headers = {'Accept': 'application/json',                  # retrieve with f.__annotations__
                'Content-Type': 'application/json',
                'Authorization': f'Bearer ' + token}
    params = [('limit', 50), ('offset', offset * 50)]
    try:
        response = requests.get('https://api.spotify.com/v1/me/tracks',
                                headers=headers, params=params)
        json = response.json()
        return json
    except:
        return False

In [5]:
# checks if all values in an interating object are equal to each other
def all_equal(iterator):
    iterator = iter(iterator)
    try:
        first = next(iterator)
    except StopIteration:
        return True
    return all(first == x for x in iterator)

# checks if len of values for all keys are equal to each other in a dictionary
def dict_vals_len_equal(dict):
    val_len = []
    for key in dict:
        val_len.append(len(dict[key]))
    return all_equal(val_len)

In [6]:
# get saved songs info in a dictionary
def get_saved_songs_dict(token: str):

    song_dict = {'id':[],
                 'name':[],
                 'album':[],
                 'artist':[],
                 'popularity':[],
                 'year_added':[],
                 'month_added':[]}
    
    offset = 0
    more_songs = True
    while(more_songs):
        
        saved_songs = get_saved_songs(token, offset)

        if len(saved_songs['items']) == 0:
            more_songs = False
        for item in saved_songs['items']:
             # track id
            track_id = item['track']['id']
            song_dict['id'].append(track_id)

            # added date/time
            track_added_at = item['added_at']
            year_month = track_added_at.split('-')
            song_dict['year_added'].append(year_month[0])
            song_dict['month_added'].append(year_month[1])

            # track name
            track_name = item['track']['name']
            song_dict['name'].append(track_name)

            # album name
            album_name = item['track']['album']['name']
            song_dict['album'].append(album_name)

            # artist
            artist = item['track']['artists'][0]['name']
            song_dict['artist'].append(artist)

            # popularity
            popularity = item['track']['popularity']
            song_dict['popularity'].append(popularity)
        offset += 1
    
    if dict_vals_len_equal(song_dict):
        return song_dict
    else:
        return "Not all values are equal lengths"

In [7]:
song_dict = get_saved_songs_dict(token)

In [8]:
features_dict = {'danceability':[],
            'energy':[],
            'key':[],
            'loudness':[],
            'mode':[],
            'speechiness':[],
            'acousticness':[],
            'instrumentalness':[],
            'liveness':[],
            'valence':[],
            'tempo':[],
            'time_signature':[]}

ids = song_dict['id']

for song_id in ids:
    song_features = sp.audio_features(song_id)[0]
    for feature in features_dict:
        features_dict[feature].append(song_features[feature])

In [9]:
# faster if I get audio_features 50 songs at a time?

def get_features(ids):
    features_dict = {'danceability':[],
            'energy':[],
            'key':[],
            'loudness':[],
            'mode':[],
            'speechiness':[],
            'acousticness':[],
            'instrumentalness':[],
            'liveness':[],
            'valence':[],
            'tempo':[],
            'time_signature':[]}

    for song_id in ids:
        song_features = sp.audio_features(song_id)[0]
        for feature in features_dict:
            features_dict[feature].append(song_features[feature])
            
    if dict_vals_len_equal(features_dict):
        return features_dict
    else:
        return "Not all values are equal lengths"

In [10]:
features_dict = get_features(song_dict['id'])

In [11]:
# concatenate saved songs dict with the features dict
saved_songs_dict = dict(song_dict, **features_dict)

In [12]:
# convert to panda dataframe and then save
saved_songs_df = pd.DataFrame.from_dict(saved_songs_dict)
saved_songs_df.to_pickle("./saved_songs_df.pkl")